In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditcardfraud/creditcard.csv


In [2]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
#  loading the dataset to a pandas DataFrame
df = pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")

In [4]:
df.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
df.shape


(284807, 31)

In [6]:
# dataset informations
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [7]:
# checking  the number of  missing values in each column
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [8]:
# distribution of legit transactions & fraudulent transactions
df["Class"].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [9]:
#  separating the data for analysis
legit = df[df.Class == 0]
fraud = df[df.Class == 1]

In [10]:
print(f"legit: {legit.shape}")
print(f"fraud : {fraud.shape}")

legit: (284315, 31)
fraud : (492, 31)


In [11]:
legit.Amount.describe()


count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [12]:
fraud.Amount.describe()


count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [13]:
#  compare the values for both transactions
df.groupby("Class").mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [15]:
legit_sample = legit.sample(n=492)


**Concatenating two DataFrames¶**
****

In [16]:
new_df = pd.concat([legit_sample,fraud],axis = 0)


In [17]:
new_df.head(5)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
188938,128199.0,-1.298083,1.038356,0.123204,-1.077794,0.127203,-0.300274,1.661772,0.043134,0.078615,...,-0.272098,-0.738800,-0.265395,0.590385,0.818700,-0.115080,-0.362558,-0.253949,178.98,0
90341,62979.0,-0.304761,0.775504,1.495899,1.245456,0.443938,0.250219,0.271509,0.300723,-0.831822,...,0.123985,0.157014,0.155112,-0.422953,-1.255026,-0.477301,0.216708,0.235533,15.59,0
45309,42288.0,-4.421848,-1.717321,1.256345,1.907431,1.028951,-0.663200,-1.416324,-2.210809,-0.796283,...,-1.872503,0.513793,2.674069,0.517517,0.969143,-0.307191,0.622243,-0.473614,29.00,0
150448,93369.0,1.926916,-0.704980,-0.438290,-1.307663,-0.845758,-0.801194,-0.655904,-0.188997,3.010668,...,0.165486,0.902657,0.086229,0.038569,-0.178876,-0.207509,0.001611,-0.049383,49.73,0
186425,127102.0,1.719904,-0.621846,-2.459654,0.097367,0.785759,-0.300907,0.701264,-0.230983,0.136772,...,0.142863,-0.020132,-0.167612,-0.112047,0.260581,-0.295311,-0.094901,-0.042655,206.38,0


In [18]:
new_df["Class"].value_counts()


Class
0    492
1    492
Name: count, dtype: int64

In [19]:
new_df.groupby("Class").mean()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,96126.310976,-0.023873,-0.104302,0.019289,0.013865,-0.123116,-0.035433,0.095728,0.010045,-0.038817,...,0.015410,0.012615,-0.026563,-0.026636,0.055508,-0.013946,0.008124,0.007443,-0.000807,119.403333
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [20]:
x = new_df.drop(columns=["Class"], axis = 1)
y = new_df["Class"]

In [23]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2, stratify = y, random_state = 42)

In [24]:
x_train.shape


(787, 30)

In [25]:
model = LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [26]:
#  accuracy on training data
y_pred = model.predict(x_train)
data_accuracy = accuracy_score(y_pred,y_train)

In [27]:
print(f" Accuracy on training data:> {data_accuracy}")


 Accuracy on training data:> 0.9415501905972046


In [28]:
#  accuracy on training data
y_pred = model.predict(x_test)
data_accuracy = accuracy_score(y_pred,y_test)

In [29]:
print(f" Accuracy on training data:> {data_accuracy}")


 Accuracy on training data:> 0.9543147208121827
